In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import ascii

indat = ascii.read("DiF_in.csv", format='csv')

mu = 18.477

Rg = 3.518
Rr = 2.617
Ri = 1.971

logP = np.log10(indat['period'])
FeH = indat['FeH']
BV = indat['Bmag'] - indat['Vmag']

gmag = indat['Bmag'] - 0.108 - 0.485*BV - 0.032*BV*BV
rmag = indat['Vmag'] + 0.082 - 0.462*BV + 0.041*BV*BV
imag = indat['Imag'] + 0.341 + 0.154*BV - 0.025*BV*BV

g0 = gmag - Rg*indat['EBV'] - mu
r0 = rmag - Rr*indat['EBV'] - mu
i0 = imag - Ri*indat['EBV'] - mu

In [3]:
import pandas as pd

from astropy.io import ascii

from astroquery.vizier import Vizier
vizier = Vizier()

def query(catalog, vtype):
    vizier  = Vizier(columns=["[DCM2005]","RAJ2000", "DEJ2000", "VType", "Per","<Bint>", "<Vint>", "<Iint>","Notes"], 
                     column_filters={"VType":vtype})
    vizier.ROW_LIMIT = -1
    results = vizier.get_catalogs(catalog)[0]
    return results

type_ab = query("J/A+A/430/603/variables", "ab")
type_c  = query("J/A+A/430/603/variables", "c")

In [4]:
Ng22_PLZcoeffs = {
    'g': {'RR0':     {'a':  0.649, 'b': -0.302, 'c': 0.159}, 
          'RR1':     {'a':  0.411, 'b': -0.342, 'c': 0.092},
          'RR0+RR1': {'a':  0.801, 'b': -0.032, 'c': 0.190}},
    'r': {'RR0':     {'a':  0.337, 'b': -1.090, 'c': 0.139}, 
          'RR1':     {'a': -0.082, 'b': -1.393, 'c': 0.091},
          'RR0+RR1': {'a':  0.432, 'b': -0.874, 'c': 0.154}},
    'i': {'RR0':     {'a':  0.243, 'b': -1.432, 'c': 0.144}, 
          'RR1':     {'a': -0.205, 'b': -1.706, 'c': 0.077},
          'RR0+RR1': {'a':  0.249, 'b': -1.362, 'c': 0.115}}
}

Na24_PLcoeffs01 = { # Parallax
    'g': {'RR0':     {'a':  -2.242, 'b': 0.805, 'c': 0}, 
          'RR1':     {'a':  -1.984, 'b': 0.605, 'c': 0},
          'RR0+RR1': {'a':  -1.500, 'b': 0.773, 'c': 0}},
    'r': {'RR0':     {'a':  -2.528, 'b': 0.660, 'c': 0}, 
          'RR1':     {'a':  -2.266, 'b': 0.557, 'c': 0},
          'RR0+RR1': {'a':  -1.929, 'b': 0.634, 'c': 0}},
    'i': {'RR0':     {'a':  -2.799, 'b': 0.642, 'c': 0}, 
          'RR1':     {'a':  -2.393, 'b': 0.598, 'c': 0},
          'RR0+RR1': {'a':  -2.249, 'b': 0.619, 'c': 0}}
}

Na24_PLcoeffs02 = { # ABL
    'g': {'RR0':     {'a':  -2.318, 'b': 0.810, 'c': 0}, 
          'RR1':     {'a':  -1.862, 'b': 0.615, 'c': 0},
          'RR0+RR1': {'a':  -1.506, 'b': 0.783, 'c': 0}},
    'r': {'RR0':     {'a':  -2.594, 'b': 0.663, 'c': 0}, 
          'RR1':     {'a':  -2.143, 'b': 0.566, 'c': 0},
          'RR0+RR1': {'a':  -1.918, 'b': 0.641, 'c': 0}},
    'i': {'RR0':     {'a':  -2.870, 'b': 0.644, 'c': 0}, 
          'RR1':     {'a':  -2.280, 'b': 0.606, 'c': 0},
          'RR0+RR1': {'a':  -2.230, 'b': 0.626, 'c': 0}}
}
Na24_PLcoeffs03 = { # Geometric Distance
    'g': {'RR0':     {'a':  -2.162, 'b': 0.802, 'c': 0}, 
          'RR1':     {'a':  -1.983, 'b': 0.604, 'c': 0},
          'RR0+RR1': {'a':  -1.448, 'b': 0.771, 'c': 0}},
    'r': {'RR0':     {'a':  -2.447, 'b': 0.657, 'c': 0}, 
          'RR1':     {'a':  -2.266, 'b': 0.556, 'c': 0},
          'RR0+RR1': {'a':  -1.878, 'b': 0.633, 'c': 0}},
    'i': {'RR0':     {'a':  -2.718, 'b': 0.639, 'c': 0}, 
          'RR1':     {'a':  -2.392, 'b': 0.597, 'c': 0},
          'RR0+RR1': {'a':  -2.197, 'b': 0.618, 'c': 0}}
}
Na24_PLcoeffs04 = { # Photo-geometric distance
    'g': {'RR0':     {'a':  -2.179, 'b': 0.803, 'c': 0}, 
          'RR1':     {'a':  -2.035, 'b': 0.607, 'c': 0},
          'RR0+RR1': {'a':  -1.462, 'b': 0.771, 'c': 0}},
    'r': {'RR0':     {'a':  -2.465, 'b': 0.658, 'c': 0}, 
          'RR1':     {'a':  -2.317, 'b': 0.558, 'c': 0},
          'RR0+RR1': {'a':  -1.891, 'b': 0.633, 'c': 0}},
    'i': {'RR0':     {'a':  -2.736, 'b': 0.640, 'c': 0}, 
          'RR1':     {'a':  -2.444, 'b': 0.599, 'c': 0},
          'RR0+RR1': {'a':  -2.211, 'b': 0.618, 'c': 0}}
}

Na24_PLZcoeffs01 = { # Parallax
    'g': {'RR0':     {'a':  -0.527, 'b': 0.794, 'c': 0.264}, 
          'RR0+RR1': {'a':  -0.284, 'b': 0.791, 'c': 0.289}},
    'r': {'RR0':     {'a':  -1.230, 'b': 0.651, 'c': 0.205}, 
          'RR0+RR1': {'a':  -1.017, 'b': 0.650, 'c': 0.228}},
    'i': {'RR0':     {'a':  -1.682, 'b': 0.635, 'c': 0.174}, 
          'RR0+RR1': {'a':  -1.469, 'b': 0.633, 'c': 0.198}}
}

Na24_PLZcoeffs02 = { # ABL
    'g': {'RR0':     {'a':  -0.503, 'b': 0.798, 'c': 0.266}, 
          'RR0+RR1': {'a':  -0.290, 'b': 0.795, 'c': 0.290}},
    'r': {'RR0':     {'a':  -1.227, 'b': 0.655, 'c': 0.204}, 
          'RR0+RR1': {'a':  -1.031, 'b': 0.652, 'c': 0.227}},
    'i': {'RR0':     {'a':  -1.673, 'b': 0.638, 'c': 0.175}, 
          'RR0+RR1': {'a':  -1.479, 'b': 0.635, 'c': 0.197}}
}

Na24_PLZcoeffs03 = { # Geometric Distance
    'g': {'RR0':     {'a':  -0.389, 'b': 0.792, 'c': 0.267}, 
          'RR0+RR1': {'a':  -0.217, 'b': 0.790, 'c': 0.285}},
    'r': {'RR0':     {'a':  -1.092, 'b': 0.650, 'c': 0.208}, 
          'RR0+RR1': {'a':  -0.950, 'b': 0.648, 'c': 0.223}},
    'i': {'RR0':     {'a':  -1.544, 'b': 0.633, 'c': 0.177}, 
          'RR0+RR1': {'a':  -1.402, 'b': 0.632, 'c': 0.193}}
}

Na24_PLZcoeffs04 = { # Geometric Distance
    'g': {'RR0':     {'a':  -0.473, 'b': 0.791, 'c': 0.257}, 
          'RR0+RR1': {'a':  -0.260, 'b': 0.789, 'c': 0.280}},
    'r': {'RR0':     {'a':  -1.177, 'b': 0.650, 'c': 0.198}, 
          'RR0+RR1': {'a':  -0.993, 'b': 0.647, 'c': 0.218}},
    'i': {'RR0':     {'a':  -1.629, 'b': 0.633, 'c': 0.168}, 
          'RR0+RR1': {'a':  -1.445, 'b': 0.631, 'c': 0.188}}
}

P0_pivot = {'RR0':     -0.25,
            'RR1':     -0.45,
            'RR0+RR1': -0.25}

In [5]:
Ng22_PWZcoeffs = {
    'Wgr': {'RR0':     {'a': -0.644, 'b': -3.324, 'c': 0.052}, 
            'RR1':     {'a': -1.327, 'b': -3.680, 'c': 0.013},
            'RR0+RR1': {'a': -0.727, 'b': -3.286, 'c': 0.010}},
    'Wri': {'RR0':     {'a':  0.093, 'b': -2.600, 'c': 0.193}, 
            'RR1':     {'a': -0.884, 'b': -3.503, 'c': 0.067},
            'RR0+RR1': {'a':  0.010, 'b': -2.756, 'c': 0.149}},
    'Wgi': {'RR0':     {'a': -0.198, 'b': -2.908, 'c': 0.142}, 
            'RR1':     {'a': -0.205, 'b': -3.696, 'c': 0.055},
            'RR0+RR1': {'a':  0.249, 'b': -3.066, 'c': 0.101}}
}
Na24_PWcoeffs01 = { # Parallax
    'Wgr': {'RR0':     {'a':  -3.626, 'b': 0.587, 'c': 0}, 
            'RR1':     {'a':  -2.780, 'b': 0.723, 'c': 0},
            'RR0+RR1': {'a':  -3.226, 'b': 0.574, 'c': 0}},
    'Wri': {'RR0':     {'a':  -3.357, 'b': 0.239, 'c': 0}, 
            'RR1':     {'a':  -3.086, 'b': 0.415, 'c': 0},
            'RR0+RR1': {'a':  -3.175, 'b': 0.232, 'c': 0}},
    'Wgi': {'RR0':     {'a':  -3.508, 'b': 0.434, 'c': 0}, 
            'RR1':     {'a':  -2.914, 'b': 0.588, 'c': 0},
            'RR0+RR1': {'a':  -3.157, 'b': 0.424, 'c': 0}}
}
Na24_PWcoeffs02 = { # ABL
    'Wgr': {'RR0':     {'a':  -3.700, 'b': 0.587, 'c': 0}, 
            'RR1':     {'a':  -2.707, 'b': 0.729, 'c': 0},
            'RR0+RR1': {'a':  -3.171, 'b': 0.580, 'c': 0}},
    'Wri': {'RR0':     {'a':  -3.395, 'b': 0.241, 'c': 0}, 
            'RR1':     {'a':  -2.977, 'b': 0.423, 'c': 0},
            'RR0+RR1': {'a':  -3.146, 'b': 0.237, 'c': 0}},
    'Wgi': {'RR0':     {'a':  -3.561, 'b': 0.435, 'c': 0}, 
            'RR1':     {'a':  -2.827, 'b': 0.594, 'c': 0},
            'RR0+RR1': {'a':  -3.127, 'b': 0.422, 'c': 0}}
}
Na24_PWcoeffs03 = { # Geometric Distance
    'Wgr': {'RR0':     {'a':  -3.546, 'b': 0.584, 'c': 0}, 
            'RR1':     {'a':  -2.779, 'b': 0.722, 'c': 0},
            'RR0+RR1': {'a':  -3.174, 'b': 0.572, 'c': 0}},
    'Wri': {'RR0':     {'a':  -3.277, 'b': 0.237, 'c': 0}, 
            'RR1':     {'a':  -3.085, 'b': 0.414, 'c': 0},
            'RR0+RR1': {'a':  -3.123, 'b': 0.231, 'c': 0}},
    'Wgi': {'RR0':     {'a':  -3.427, 'b': 0.432, 'c': 0}, 
            'RR1':     {'a':  -2.913, 'b': 0.587, 'c': 0},
            'RR0+RR1': {'a':  -3.151, 'b': 0.423, 'c': 0}}
}
Na24_PWcoeffs04 = { # Photo-geometric distance
    'Wgr': {'RR0':     {'a':  -3.563, 'b': 0.584, 'c': 0}, 
            'RR1':     {'a':  -2.830, 'b': 0.725, 'c': 0},
            'RR0+RR1': {'a':  -3.185, 'b': 0.572, 'c': 0}},
    'Wri': {'RR0':     {'a':  -3.294, 'b': 0.237, 'c': 0}, 
            'RR1':     {'a':  -3.137, 'b': 0.416, 'c': 0},
            'RR0+RR1': {'a':  -3.083, 'b': 0.225, 'c': 0}},
    'Wgi': {'RR0':     {'a':  -3.445, 'b': 0.432, 'c': 0}, 
            'RR1':     {'a':  -2.965, 'b': 0.590, 'c': 0},
            'RR0+RR1': {'a':  -3.114, 'b': 0.417, 'c': 0}}
}

Na24_PWZcoeffs01 = { # Parallax
    'Wgr': {'RR0':     {'a':  -3.061, 'b': 0.584, 'c': 0.082}, 
            'RR0+RR1': {'a':  -2.848, 'b': 0.583, 'c': 0.105}},
    'Wri': {'RR0':     {'a':  -3.273, 'b': 0.239, 'c': 0.033}, 
            'RR0+RR1': {'a':  -3.148, 'b': 0.238, 'c': 0.050}},
    'Wgi': {'RR0':     {'a':  -3.154, 'b': 0.433, 'c': 0.060}, 
            'RR0+RR1': {'a':  -2.979, 'b': 0.432, 'c': 0.081}}
}

Na24_PWZcoeffs02 = { # ABL
    'Wgr': {'RR0':     {'a':  -3.027, 'b': 0.587, 'c': 0.084}, 
            'RR0+RR1': {'a':  -2.837, 'b': 0.585, 'c': 0.107}},
    'Wr:': {'RR0':     {'a':  -3.347, 'b': 0.241, 'c': 0.019}, 
            'RR0+RR1': {'a':  -3.178, 'b': 0.240, 'c': 0.042}},
    'Wg:': {'RR0':     {'a':  -3.169, 'b': 0.435, 'c': 0.055}, 
            'RR0+RR1': {'a':  -2.987, 'b': 0.433, 'c': 0.078}}
}
Na24_PWZcoeffs03 = { # Geometric Distance
    'Wgr': {'RR0':     {'a':  -2.923, 'b': 0.583, 'c': 0.085}, 
            'RR0+RR1': {'a':  -2.824, 'b': 0.580, 'c': 0.096}},
    'Wri': {'RR0':     {'a':  -3.135, 'b': 0.237, 'c': 0.036}, 
            'RR0+RR1': {'a':  -3.124, 'b': 0.236, 'c': 0.040}},
    'Wgi': {'RR0':     {'a':  -3.016, 'b': 0.431, 'c': 0.063}, 
            'RR0+RR1': {'a':  -2.955, 'b': 0.429, 'c': 0.071}}
}
Na24_PWZcoeffs04 = { # Geometric Distance
    'Wgr': {'RR0':     {'a':  -3.008, 'b': 0.582, 'c': 0.075}, 
            'RR0+RR1': {'a':  -2.835, 'b': 0.580, 'c': 0.095}},
    'Wri': {'RR0':     {'a':  -3.220, 'b': 0.236, 'c': 0.026}, 
            'RR0+RR1': {'a':  -3.135, 'b': 0.236, 'c': 0.039}},
    'Wgi': {'RR0':     {'a':  -3.101, 'b': 0.430, 'c': 0.054}, 
            'RR0+RR1': {'a':  -2.967, 'b': 0.430, 'c': 0.070}}
}

In [6]:
def relation(logp, feh, band, rrtype, coeffs, model):
    a, b, c = coeffs[band][rrtype].values()
    P = np.linspace(min(logp), max(logp), 200)
    if model == 'Ng22':
        return a + b * P + c * feh, P
    elif model == 'Na24':
        P0 = P0_pivot[rrtype]
        feh0 = -1.5
        return a * (P - P0) + b + c * (feh - feh0), P

In [51]:
mi = indat['Imag'] < 99.999

mab = indat['rtype'] == 'ab'
mc  = indat['rtype'] == 'c'

ilogP = logP[mi]
iFeH = FeH[mi]
ii0 = i0[mi]
iab = mab[mi]
ic  = mc[mi]

bands = ['g', 'r', 'i']
PLZcolors = {'g': ['#1900FF', '#3ED5FF'], 'r': ['green', '#1AFF00'], 'i': ['#922017', '#FF4600']}
ylabel_map = {'g': r"$M_g$ [mag]", 'r': r"$M_r$ [mag]", 'i': r"$M_i$ [mag]"}

RRab_gri = {
    'g': (logP[mab], g0[mab]), 
    'r': (logP[mab], r0[mab]), 
    'i': (ilogP[iab], ii0[iab])
}
RRc_gri = {
    'g': (logP[mc], g0[mc]), 
    'r': (logP[mc], r0[mc]), 
    'i': (ilogP[ic], ii0[ic])
}

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(8, 12), sharex=True)

axs[0].set_title("RRab PLZ Relations", size=18)

for i, band in enumerate(bands):
    logP_values, mag_values = RRab_gri[band]
    
    Ng22, NgP = relation(logP_values, -1.48, band, 'RR0', Ng22_PLZcoeffs, 'Ng22')
    Na24, NaP = relation(logP_values, -1.48, band, 'RR0', Na24_PLZcoeffs01, 'Na24')

    axs[i].scatter(logP_values, mag_values, color=PLZcolors[band][0])
    axs[i].plot(NgP, Ng22, 'orange', label="Ngeow+ 2022")
    axs[i].plot(NaP, Na24, 'mediumturquoise', label="Narloch+ 2024")
    axs[i].minorticks_on()
    axs[i].invert_yaxis()
    axs[i].set_ylabel(ylabel_map[band], size=12)
    
    if i == 0:
        axs[i].legend(loc='best')

axs[-1].set_xlabel(r"$\log P$", size=12)
plt.tight_layout()
plt.show()
# plt.savefig('DiF-RRab_PLZ.pdf', dpi=300)

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(8, 12), sharex=True)

axs[0].set_title("RRab+RRc PLZ Relations", size=18)

for i, band in enumerate(bands):
    logPab_values, magab_values = RRab_gri[band]
    logPc_values,  magc_values  = RRc_gri[band]
    logPc_values = logPc_values + 0.127
    
    logP_values = list(logPab_values) + list(logPc_values)
    Ng22, NgP = relation(logP_values, -1.48, band, 'RR0+RR1', Ng22_PLZcoeffs, 'Ng22')
    Na24, NaP = relation(logP_values, -1.48, band, 'RR0+RR1', Na24_PLZcoeffs01, 'Na24')

    axs[i].scatter(logPab_values, magab_values, color=PLZcolors[band][0])
    axs[i].scatter(logPc_values,  magc_values, color=PLZcolors[band][1])
    axs[i].plot(NgP, Ng22, 'orange', label="Ngeow+ 2022")
    axs[i].plot(NaP, Na24, 'mediumturquoise', label="Narloch+ 2024")

    axs[i].minorticks_on()
    axs[i].invert_yaxis()
    axs[i].set_ylabel(ylabel_map[band], size=12)
    
    if i == 0:
        axs[i].legend(loc='best')

axs[-1].set_xlabel(r"$\log P$", size=12)
plt.tight_layout()
plt.show()
# plt.savefig('DiF-RRabc_PLZ.pdf', dpi=300)

In [141]:
w = ['Wgr', 'Wri', 'Wgi']
PWZcolors = {}